In [5]:
# using pandas to read csv's and join the tables b4 parsing to ideal format
import pandas as pd

splitset_df = pd.read_csv("datasetSplit.txt", sep=",")
splitset_df

,sentence_index,splitset_label
0,1,1
1,2,1
2,3,2
3,4,2
4,5,2
...,...,...
11850,11851,1
11851,11852,1
11852,11853,1
11853,11854,1


In [6]:
sentences_df = pd.read_csv("datasetSentences.txt", sep="\t")
sentences_df

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,"Emerges as something rare , an issue movie tha..."
...,...,...
11850,11851,A real snooze .
11851,11852,No surprises .
11852,11853,We 've seen the hippie-turned-yuppie plot befo...
11853,11854,Her fans walked out muttering words like `` ho...


In [7]:
sentiment_df = pd.read_csv("sentiment_labels.txt", sep="|")
sentiment_df

,sentence_index,sentiment_values
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708
...,...,...
239227,239227,0.36111
239228,239228,0.38889
239229,239229,0.33333
239230,239230,0.88889


In [8]:
split_df = pd.read_csv("datasetSplit.txt", sep=",")
split_df

,sentence_index,splitset_label
0,1,1
1,2,1
2,3,2
3,4,2
4,5,2
...,...,...
11850,11851,1
11851,11852,1
11852,11853,1
11853,11854,1


In [9]:
merged_df = pd.merge(sentences_df, split_df, on='sentence_index')
merged_df = pd.merge(merged_df, sentiment_df, on='sentence_index')

merged_df

,sentence_index,sentence,splitset_label,sentiment_values
0,1,The Rock is destined to be the 21st Century 's...,1,0.50000
1,2,The gorgeously elaborate continuation of `` Th...,1,0.44444
2,3,Effective but too-tepid biopic,2,0.50000
3,4,If you sometimes like to go to the movies to h...,2,0.42708
4,5,"Emerges as something rare , an issue movie tha...",2,0.37500
...,...,...,...,...
11850,11851,A real snooze .,1,0.19444
11851,11852,No surprises .,1,0.61111
11852,11853,We 've seen the hippie-turned-yuppie plot befo...,1,0.72222
11853,11854,Her fans walked out muttering words like `` ho...,1,0.50000


In [13]:
max([len(x) for x in merged_df['sentence']])

283

In [6]:

# just so I get consistent ordering here
COLUMNS = list(sorted(
    k for k in merged_df.keys() if k != "splitset_label" and k != "sentence_index"
))
print(COLUMNS)

['sentence', 'sentiment_values']


In [45]:
import tiktoken
import numpy as np
gpt2_encoder = tiktoken.get_encoding("gpt2")

PADDING_VALUE = gpt2_encoder.max_token_value + 2

def to_label(sentiment):
    """       sentiment_labels.txt contains all phrase ids and the corresponding sentiment labels, separated by a vertical line.
    Note that you can recover the 5 classes by mapping the positivity probability using the following cut-offs:
    [0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.6, 0.8], (0.8, 1.0]
    """
    if sentiment <= 0.2:
        return 0
    elif sentiment <= 0.4:
        return 1
    elif sentiment <= 0.6:
        return 2
    elif sentiment <= 0.8:
        return 3
    else:
        return 4



def to_tensor(split):
    split_number = to_split_number(split)
    dataset = merged_df[merged_df['splitset_label'] == split_number].to_dict()
    culumns = list(dataset.keys())
    dataset_as_map = {}
    for column in culumns:
        dataset_as_map[column] = list(dataset[column].values())
    
    dataset_as_map["sentence"] = [gpt2_encoder.encode(sentence) for sentence in dataset_as_map["sentence"]]

    
    size = len(dataset_as_map["sentence"])

    dataset_as_list = []


    sentences = []
    sentiments = []
    for i in range(size):

        sentence = dataset_as_map["sentence"][i]

        if len(sentence) < 100:
            sentence = sentence + [PADDING_VALUE] * (100 - len(sentence))


        sentences.append(sentence)


        sentiment = dataset_as_map["sentiment_values"][i]

        sentiment = to_label(sentiment)
        sentiments.append(sentiment)



    return np.array(sentences), np.array(sentiments)


def to_split_number(split):
    """ 	1 = train
    2 = test
    3 = dev """

    if split == "train":
        return 1
    elif split == "test":
        return 2
    elif split == "dev":
        return 3
    else:
        raise RuntimeError("Invalid split value")

def to_split_label(split):
    """ 	1 = train
	2 = test
	3 = dev """

    if split == 1:
        return "train"
    elif split == 2:
        return "test"
    elif split == 3:
        return "dev"
    else:
        raise RuntimeError("Invalid split value")



In [44]:
dev = to_tensor("dev")


for x in dev:

    if len(x[1]) != 100:
        print(x[0])
        print(len(x[0]))
        print("")


TypeError: object of type 'float' has no len()

In [46]:

dataset = {
    "train": to_tensor("train"),
    "test": to_tensor("test"),
    "dev": to_tensor("dev")
}


import pickle
with open("stanfordSentimentTreebank.pickle", "wb") as f:
    pickle.dump(dataset, f)


In [34]:
dataset["dev"].shape

AttributeError: 'list' object has no attribute 'shape'

dataset

dataset